#**Purpose of this Analysis**


The dataset consists of customer information over the month of January 2020 for a company. The purpose of this analysis is to determine the key features that lead to which wireless carrier employees will likely select. 

#Build Dask environment

In [1]:
pip install --upgrade 'dask[complete]'

     |████████████████████████████████| 849kB 2.7MB/s 
     |████████████████████████████████| 92kB 4.4MB/s 
     |████████████████████████████████| 665kB 7.3MB/s 
     |████████████████████████████████| 102kB 7.3MB/s 
  Created wheel for locket: filename=locket-0.2.0-cp36-none-any.whl size=4040 sha256=44113bcb111d6818a6d9389cb78e060ad8a11870a1f151f2b63ac1d021b85e74
  Stored in directory: /root/.cache/pip/wheels/26/1e/e8/4fa236ec931b1a0cdd61578e20d4934d7bf188858723b84698
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=a772a0ca488fe4750f3b82aa22d16868e19fd87ddf3b69bb7344a57eb794973a
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built locket contextvars
ERROR: distributed 2.30.0 has requirement cloudpickle>=1.5.0, but you'll have cloudpickle 1.3.0 which is incompatible.
  Found existing installation: distributed 1.25.3
    Uninstalling distributed-1.25.3:
      Successfu

#Build ML library for Dask

In [2]:
pip install dask-ml

     |████████████████████████████████| 143kB 2.8MB/s 
     |████████████████████████████████| 6.8MB 7.9MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


#**Restart Runtime and Continue**

Restart required in order to apply several packages that were installed to this active runtime.

#Install Aiohttp

In [ ]:
pip install aiohttp

     |████████████████████████████████| 1.2MB 3.5MB/s 
     |████████████████████████████████| 266kB 17.2MB/s 
     |████████████████████████████████| 153kB 20.5MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3161 sha256=ab6520f3f9a2bafba6784df57bb089630df6a2e8fa85387c83e1612164330363
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl


#Client Dashboard

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from dask.distributed import Client, progress

client = Client(n_workers=4, threads_per_worker=2, memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:38637 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 8.00 GB


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

get_ipython().system_raw('./ngrok http 8787 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-10-18 21:12:07--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.71.209.190, 34.205.198.58, 52.2.56.23, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.71.209.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  40.3MB/s    in 0.3s    

2020-10-18 21:12:08 (40.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


#Load Data

Data loaded into a Dask dataframe, dropping columns that will not be used in analysis, and applying dummy encoder to target variable(Carrier). In this case the taret was an object type so conversion was fairly simple.

In [ ]:
import math
from scipy import stats
import datetime as dt
import time

from dask import delayed
import dask.dataframe as dd
from dask_ml.preprocessing import DummyEncoder, Categorizer
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import StandardScaler
from dask_ml.decomposition import PCA
import dask.array as da

from sklearn.metrics import roc_auc_score
import joblib
from dask_ml.model_selection import train_test_split

In [ ]:
capstone = dd.read_csv('https://raw.githubusercontent.com/r4geqwit/WirelessData/master/RawDataJanuary.csv', dtype={'Carrier': 'category'},assume_missing=True)

In [ ]:
df = capstone.drop(['Month', 'Plans_1', 'Employee Number', 'Year'], axis=1)
pipe = make_pipeline(
    Categorizer(), DummyEncoder(drop_first=True))
df1 = pipe.fit_transform(df)

#Create Train and Test splits

Aside from some changes here for use with Dask, the models all run the same with similar results to Pandas and Numpy variation of this model. 

In [ ]:
X = df1.drop('Carrier_Verizon US', axis=1)
y = df1['Carrier_Verizon US']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train.persist()
X_test.persist()
y_train.persist()
y_test.persist()

Dask Series Structure:
npartitions=1
    uint8
      ...
Name: Carrier_Verizon US, dtype: uint8
Dask Name: split, 1 tasks

#Train and test Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression(solver='sag', penalty='l2', max_iter=1000, random_state=1)
with joblib.parallel_backend('dask'):
  lr.fit(X_train.values.compute(), y_train.values.compute())

preds_train = lr.predict(X_train.values.compute())
preds_test = lr.predict(X_test.values.compute())

print("Training score is: ", roc_auc_score(preds_train, y_train.values.compute()))
print("Test score is: ", roc_auc_score(preds_test, y_test.values.compute()))

Training score is:  0.8486011057128497
Test score is:  0.8754578754578755


#Train and test KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=8)

with joblib.parallel_backend('dask'):
  knn.fit(X_train.values.compute(), y_train.values.compute())

preds_train = knn.predict(X_train.values.compute())
preds_test = knn.predict(X_test.values.compute())

print("Training score is: ", roc_auc_score(preds_train, y_train.values.compute()))
print("Test score is: ", roc_auc_score(preds_test, y_test.values.compute()))

Training score is:  0.7141466750841751
Test score is:  0.5327272727272727


#Train and test SVM

In [ ]:
from sklearn import svm

In [ ]:
time_start = time.time()

svc = svm.SVC(kernel = 'linear', random_state=1)

with joblib.parallel_backend('dask'):
  svc.fit(X_train.values.compute(), y_train.values.compute())

preds_train = svc.predict(X_train.values.compute())
preds_test = svc.predict(X_test.values.compute())

print("Training score is: ", roc_auc_score(preds_train, y_train.values.compute()))
print("Test score is: ", roc_auc_score(preds_test, y_test.values.compute()))

print('Time elapsed: {} seconds'.format(time.time()-time_start)) 

Training score is:  0.9673898475456526
Test score is:  0.9680851063829787
Time elapsed: 98.0911934375763 seconds


#Train and test Decision Tree model

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtc = DecisionTreeClassifier(
     criterion='entropy',
    max_features=5,
    max_depth=5,
    random_state=1 
)

with joblib.parallel_backend('dask'):
  dtc.fit(X_train.values.compute(), y_train.values.compute())

preds_train = dtc.predict(X_train.values.compute())
preds_test = dtc.predict(X_test.values.compute())

print("Training score is: ", roc_auc_score(preds_train, y_train.values.compute()))
print("Test score is: ", roc_auc_score(preds_test, y_test.values.compute()))

Training score is:  0.9801671197020034
Test score is:  0.9791666666666667


#Train and test Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=1000, random_state=1)
with joblib.parallel_backend('dask'):
  rfc.fit(X_train.values.compute(), y_train.values.compute())

preds_train = rfc.predict(X_train.values.compute())
preds_test = rfc.predict(X_test.values.compute())

print("Training score is: ", roc_auc_score(preds_train, y_train.values.compute()))
print("Test score is: ", roc_auc_score(preds_test, y_test.values.compute()))

Training score is:  1.0
Test score is:  1.0


#Test and train Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gbc = GradientBoostingClassifier(
    n_estimators=1000, 
    max_depth=5, 
    loss='deviance', 
    random_state=1
    )
with joblib.parallel_backend('dask'):
  gbc.fit(X_train.values.compute(), y_train.values.compute())

preds_train = gbc.predict(X_train.values.compute())
preds_test = gbc.predict(X_test.values.compute())

print("Training score is: ", roc_auc_score(preds_train, y_train.values.compute()))
print("Test score is: ", roc_auc_score(preds_test, y_test.values.compute()))

Training score is:  1.0
Test score is:  0.9893617021276595


#Convert features into Dask array, then apply Dask ml PCA to training data

The challenge here was figuring out to convert the target variable into an array. After that was done, I chose to grab the top 20 prinicipal components of the dataset.

In [ ]:
X = df1.drop('Carrier_Verizon US', axis=1)
y = df1['Carrier_Verizon US']

X = X.to_dask_array(lengths=True)

In [ ]:
pca = PCA(n_components=20, svd_solver='tsqr')
pca.fit(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1, convert_mixed_types=True)

X_train = pca.fit_transform(X_train)
X_test = pca.fit_transform(X_test)

#Retrain and test Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression(solver='sag', penalty='l2', max_iter=1000, random_state=1)
with joblib.parallel_backend('dask'):
  lr.fit(X_train.compute(), y_train.compute())

preds_train = lr.predict(X_train.compute())
preds_test = lr.predict(X_test.compute())

print("Training score is: ", roc_auc_score(preds_train, y_train.compute()))
print("Test score is: ", roc_auc_score(preds_test, y_test.compute()))

Training score is:  0.8373569435745601
Test score is:  0.7377746279234585


#Retrain and test KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=6)

with joblib.parallel_backend('dask'):
  knn.fit(X_train.compute(), y_train.compute())

preds_train = knn.predict(X_train.compute())
preds_test = knn.predict(X_test.compute())

print("Training score is: ", roc_auc_score(preds_train, y_train.compute()))
print("Test score is: ", roc_auc_score(preds_test, y_test.compute()))

Training score is:  0.7412372799224681
Test score is:  0.5077160493827161


#Retrain and test SVM

In [ ]:
from sklearn import svm

In [ ]:
time_start = time.time()
svc = svm.SVC(kernel = 'linear', random_state=1)

with joblib.parallel_backend('dask'):
  svc.fit(X_train.compute(), y_train.compute())

preds_train = svc.predict(X_train.compute())
preds_test = svc.predict(X_test.compute())

print("Training score is: ", roc_auc_score(preds_train, y_train.compute()))
print("Test score is: ", roc_auc_score(preds_test, y_test.compute()))
print('Time elapsed: {} seconds'.format(time.time()-time_start))

Training score is:  0.9657265961613787
Test score is:  0.4656895403064623
Time elapsed: 55.67445993423462 seconds


#Retrain and test Decision Tree model

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtc = DecisionTreeClassifier(
     criterion='entropy',
    max_features=5,
    max_depth=5,
    random_state=1 
)

with joblib.parallel_backend('dask'):
  dtc.fit(X_train.compute(), y_train.compute())

preds_train = dtc.predict(X_train.compute())
preds_test = dtc.predict(X_test.compute())

print("Training score is: ", roc_auc_score(preds_train, y_train.compute()))
print("Test score is: ", roc_auc_score(preds_test, y_test.compute()))

Training score is:  0.9460527550924622
Test score is:  0.6991228070175438


#Retrain and test Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=1000, random_state=1)
with joblib.parallel_backend('dask'):
  rfc.fit(X_train.compute(), y_train.compute())

preds_train = rfc.predict(X_train.compute())
preds_test = rfc.predict(X_test.compute())

print("Training score is: ", roc_auc_score(preds_train, y_train.compute()))
print("Test score is: ", roc_auc_score(preds_test, y_test.compute()))

Training score is:  1.0
Test score is:  0.7288461538461537


#Retest and train Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gbc = GradientBoostingClassifier(
    n_estimators=1000, 
    max_depth=5, 
    loss='deviance', 
    random_state=1
    )
with joblib.parallel_backend('dask'):
  gbc.fit(X_train.compute(), y_train.compute())

preds_train = gbc.predict(X_train.compute())
preds_test = gbc.predict(X_test.compute())

print("Training score is: ", roc_auc_score(preds_train, y_train.compute()))
print("Test score is: ", roc_auc_score(preds_test, y_test.compute()))

Training score is:  1.0
Test score is:  0.7435897435897436


#Conclusion

The longest model to run was th SVM model for both the PCA and non PCA variant at about 100 seconds.

None of these models were tuned much after initial parameters were set, as this is a test of applying parellization to the model, and not creating optimal model accuracy.

In terms of results:

Prior to PCA, Decisision Tree, Rndom Forest, Gradient Boosting, and SVM all performed fantastically with over 96% accuracy on the test set.

After PCA, Gradient Boosting showed good results with 74.35% test accuracy.